<a href="https://colab.research.google.com/github/Ksoro7/first_project/blob/master/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
!pip install scikit-learn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split



In [ ]:
# Encoder les colonnes catégorielles
cat_cols = ["sex", "smoker", "region"]

df_encoded = dataset.copy()
for col in cat_cols:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df_encoded[col])

df_encoded.head()

# X = features, y = expenses
X = df_encoded.drop("expenses", axis=1)
y = df_encoded["expenses"]

# Train/test split (80/20)
train_dataset, test_dataset, train_labels, test_labels = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Train set:", train_dataset.shape)
print("Test set:", test_dataset.shape)

model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)  # sortie continue
])

model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae']
)

model.summary()

history = model.fit(
    train_dataset, train_labels,
    epochs=100,
    validation_split=0.2,
    verbose=1
)


In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
# Évaluer le modèle sur le jeu de test
loss, mae = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
    print("✅ You passed the challenge. Great job!")
else:
    print("❌ The Mean Abs Error must be less than 3500. Keep trying.")

# Générer les prédictions
test_predictions = model.predict(test_dataset).flatten()

# Tracer les résultats
plt.figure(figsize=(8,6))
plt.scatter(test_labels, test_predictions, alpha=0.6)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
plt.title("Predicted vs Actual Healthcare Costs")

lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
plt.plot(lims, lims, 'r--')  # Ligne idéale (y = x)
plt.grid(True)
plt.show()